In [37]:
import json
import os

# BASE_DIR = "/home/suyash/final_repo/WikiTableQuestions"
BASE_DIR = "/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM"
with open(os.path.join(BASE_DIR, "final_dataset_filtered", "explicit_questions_data.json")) as f:
    og_explicit_data = json.load(f)

In [38]:
print(list(og_explicit_data.items())[0])

('00009b9649d0dd0a', [{'img_id': '{IMG-{HTQ-0-4-2-0}}', 'table': 'List_of_mosques_in_Afghanistan_0', 'img_string': 'herat', 'cell_string': '{IMG-{HTQ-0-4-2-0}}', 'column_header': 'City', 'cell_idx': [4, 2]}])


In [39]:
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_questions_data.json")) as f:
    new_explicit_data = json.load(f)

In [40]:
print(list(new_explicit_data.items())[0])

('00009b9649d0dd0a', {'table_id': 'List_of_mosques_in_Afghanistan_0', 'cell_indices': [[4, 2]]})


In [41]:
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "image_id_to_image_path.json")) as f:
    new_image_id_to_image_path = json.load(f)

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "image_id_to_image_path.json")) as f:
    og_image_id_to_image_path = json.load(f)

tables_dict = {}
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "tables.jsonl")) as f:
    for line in f.readlines():
        table = json.loads(line)
        tables_dict[table["table_id"]] = table

In [42]:
import re
def get_all_image_tags(text):
    pattern = r"\{IMG-\{.*?\}\}"
    matches = re.findall(pattern, text)
    return matches


In [43]:
from collections import defaultdict

final_explicit_data = defaultdict(list)



for qid in new_explicit_data:
    new_cells = new_explicit_data[qid]['cell_indices']
    new_cells = tuple([tuple(cell) for cell in new_cells])
    new_cells_set = set(new_cells)
    
    og_data = og_explicit_data[qid]
    for og_cell in og_data:

        og_cell_idx = tuple(og_cell['cell_idx'])
        if og_cell_idx in new_cells_set:
            table_id = new_explicit_data[qid]['table_id']

            table_cell = tables_dict[table_id]['table_array'][og_cell_idx[0]][og_cell_idx[1]]

            image_tags = get_all_image_tags(table_cell)
            
            img_string = og_cell['img_string']
            for img_tag in image_tags:
                if og_image_id_to_image_path[og_cell['img_id']] == new_image_id_to_image_path[img_tag]:
                    final_explicit_data[qid].append({
                        'img_id': img_tag,
                        'table': table_id,
                        'img_string': img_string,
                        'cell_idx': og_cell['cell_idx'],
                        'cell_string': table_cell,
                        'column_header': og_cell['column_header'],
                    })
                    break
    # print(new_cells)
    # for cell in new_cells:
    #     og_data = og_explicit_data[qid]
    #     for og_cell in og_data:
    #         if cell == og_cell['cell_idx']:
    #             print("YAY")
    #             final_explicit_data
    #         print(og_cell)
    # break

In [44]:
len_to_freq = {}
for qid, q_data in final_explicit_data.items():
    len_to_freq[len(q_data)] = len_to_freq.get(len(q_data), 0) + 1

In [45]:
len_to_freq

{1: 4076,
 2: 748,
 5: 106,
 8: 58,
 9: 41,
 10: 22,
 4: 158,
 3: 325,
 13: 21,
 6: 101,
 14: 14,
 15: 40,
 11: 24,
 7: 54,
 12: 22,
 24: 2,
 16: 3,
 18: 2,
 27: 2}

In [46]:
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_questions_data.json"), "w") as f:
    json.dump(final_explicit_data, f, indent=4)